In [11]:
import boto3 # Python integration with AWS.
import pandas as pd # Read csv.
from io import StringIO, BytesIO # StringIO to read csv on AWS. BytesIO to save as Parquet file to S3.
from datetime import datetime, timedelta # Date functions.

In [12]:
# Adapter Layer Functions

def read_csv_to_df(bucket, key, decoding = 'utf-8', sep = ','):
    csv_obj = bucket.Object(key=key).get().get('Body').read().decode(decoding)
    # Read csvs with StringIO (needed for AWS) and Pandas
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=sep)
    return df

def write_df_to_s3(bucket, df, key):
    out_buffer = BytesIO()
    df.to_parquet(out_buffer, index=False)
    bucket.put_object(Body=out_buffer.getvalue(), Key=key)
    return True

def return_objects(bucket, arg_date, scr_format):
    # Convert to date type and use previous day
    min_date = datetime.strptime(arg_date, scr_format).date() - timedelta(days=1)
    # List of filtered CSVs. Filter using min_date
    objects = [obj.key for obj in bucket.objects.all() if datetime.strptime(obj.key.split("/")[0], scr_format).date() >= min_date]
    return objects

In [13]:
# Application Layer Functions, ETL

def extract(bucket, objects):
    df = pd.concat([read_csv_to_df(bucket, obj) for obj in objects], ignore_index=True)
    return df

def transform_report1(df, columns, arg_date):
    # DF with filtered columns.
    df = df.loc[:, columns]
    # Filter out empty lines.
    df.dropna(inplace=True)
    # Get opening price per ISIN and day
    # Create new column with opening price.
    df['opening_price'] = df.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')
    # Get closing price per ISIN and day
    # Create new column with closing price.
    df['closing_price'] = df.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')
    # Aggregation
    df = df.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))
    # Percent Change Prev Closing
    df['prev_closing_price'] = df.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)
    df['change_prev_closing_%'] = (df['closing_price_eur'] - df['prev_closing_price']) / df['prev_closing_price'] * 100
    # Remove column and update df
    df.drop(columns=['prev_closing_price'], inplace=True)
    # Round everything.
    df = df.round(decimals=2)
    # Filter by date, don't show previous date that was used for calculation.
    df = df[df.Date >= arg_date]
    return df

def load(bucket, df, trg_key, trg_format):
    # Write to S3.
    key = trg_key + datetime.today().strftime("%Y%m%d_%H%M%S") + trg_format
    write_df_to_s3(bucket, df, key)
    return True

# Combine all three ETL functions.
def etl_report1(scr_bucket, trg_bucket, objects, columns, arg_date, trg_key, trg_format):
    df = extract(scr_bucket, objects)
    df = transform_report1(df, columns, arg_date)
    load(trg_bucket, df, trg_key, trg_format)
    return True

In [14]:
# main function entrypoint

def main():
    # Parameters/Configurations
    # Later read config
    # Argument Date for start of report
    arg_date = '2022-11-23'
    # Moving all arguments and parameters to the beggining of the code
    scr_format = '%Y-%m-%d'
    scr_bucket = 'xetra-1234'
    trg_bucket = 'manoetl123'
    columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
    trg_key = 'xetra_daily_report_'
    trg_format = '.parquet'
    
    # Init
    # Variables for data on AWS S3.
    s3 = boto3.resource('s3')
    bucket_scr = s3.Bucket(scr_bucket)
    bucket_trg = s3.Bucket(trg_bucket)
    
    # run application
    objects = return_objects(bucket_scr, arg_date, scr_format)
    etl_report1(bucket_scr, bucket_trg, objects, columns, arg_date, trg_key, trg_format)

In [15]:
# run

main()

## Read the uploaded file

In [17]:
trg_bucket = 'manoetl123'
s3 = boto3.resource('s3')
bucket_trg = s3.Bucket(trg_bucket)

# Read the uploaded file
for obj in bucket_trg.objects.all():
    print(obj.key)

xetra_daily_report_20221122_222801.parquet
xetra_daily_report_20221122_233442.parquet
xetra_daily_report_20221123_183552.parquet


In [18]:
prq_obj = bucket_trg.Object(key='xetra_daily_report_20221122_233442.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [19]:
df_report

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT000000STR1,2022-11-22,36.40,36.85,36.40,36.85,1475,-1.60
1,AT00000FACC2,2022-11-22,7.89,8.47,7.89,8.47,1956,1.80
2,AT0000606306,2022-11-22,21.36,21.78,21.20,22.62,61546,-8.26
3,AT0000609607,2022-11-22,12.20,12.30,12.14,12.32,930,-1.28
4,AT0000644505,2022-11-22,98.40,100.80,98.40,102.20,2413,-1.56
...,...,...,...,...,...,...,...,...
3248,XS2284324667,2022-11-22,38.72,38.33,38.27,39.07,4245,0.64
3249,XS2314659447,2022-11-22,8.74,8.76,8.71,8.81,133,0.22
3250,XS2314660700,2022-11-22,22.10,22.04,22.04,22.10,239,1.60
3251,XS2376095068,2022-11-22,32.47,33.04,32.47,33.22,758,-3.36
